In [581]:
import random
from names import get_full_name
from tqdm import tqdm

class competition:
    
    def __init__(self, nr_players):
        self.player_poule = [player(skill_level = random.random(), name = get_full_name()) for i in range(nr_players)]
        
    def play_rand_matches(self, nr_of_games):
        for game in tqdm(range(nr_of_games)):
            players = random.sample(range(0, len(self.player_poule)), 2)
            
            m = match(self.player_poule[players[0]], self.player_poule[players[1]])
            m.simulate_match()
        
    def show_standings(self):
        sorted_poule = sorted(self.player_poule, key=lambda x: x.rating, reverse=True)
        print("leaderboard standings")
        print("------------------------------------")
        for p in sorted_poule[0:20]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        for p in sorted_poule[-1:-21]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        print("------------------------------------\n")
    

class match:
    
    def __init__(self, player1, player2, k_factor = 32):
        self.p1 = player1
        self.p2 = player2
        self.k_factor = k_factor
        
        self.es_p1 = 1 / (1 + 10**((player2.rating - player1.rating) / 400))
        self.es_p2 = 1 / (1 + 10**((player1.rating - player2.rating) / 400))
    
    def simulate_match(self):
        sl_p1 = self.p1.skill_level
        sl_p2 = self.p2.skill_level
        draw_odds = (1 - abs(sl_p1 - sl_p2)) * (sl_p1 + sl_p2)
        win_chances = [sl_p1, draw_odds, sl_p2]
        
        result_p1 = random.choices([1, 0.5, 0], weights = win_chances, k=1)[0]
        result_p2 = 1-result_p1
        
        self.p1.update_rating(self.k_factor, self.es_p1, result_p1)
        self.p2.update_rating(self.k_factor, self.es_p2, result_p2)
        
        return result_p1

class player:
    
    def __init__(self, rating = 1200, skill_level = 0.5, name = "filler name"):
        self.name = name
        self.rating = rating
        self.skill_level = skill_level
    
    def update_rating(self, k_factor, expected_score, result):
        change = k_factor * (result - expected_score)
        self.rating += round(change)


In [583]:
c = competition(2000)
c.show_standings()
c.play_rand_matches(100000000)
c.show_standings()

leaderboard standings
------------------------------------
Carolyn Schmuff       | 1200 | 0.315
Thomas Hamilton       | 1200 | 0.364
Eleanor Cunningham    | 1200 | 0.247
Charlene Martindale   | 1200 | 0.087
John Lane             | 1200 | 0.081
Dwayne Eversole       | 1200 | 0.760
Eugene Jimenez        | 1200 | 0.860
Sheri Swearingen      | 1200 | 0.662
Juan Waits            | 1200 | 0.043
Thomas Hohlstein      | 1200 | 0.757
Liz Fowler            | 1200 | 0.669
Lillian Williamson    | 1200 | 0.185
Shannon Mcdonough     | 1200 | 0.995
Katherine Tieng       | 1200 | 0.158
Andrea Richardson     | 1200 | 0.560
Douglas Kees          | 1200 | 0.419
Rita Whitlatch        | 1200 | 0.421
Arlene Bird           | 1200 | 0.661
Tonya Standridge      | 1200 | 0.333
Susan Weber           | 1200 | 0.107
------------------------------------



100%|████████████████████████████████████████████████████████████████| 100000000/100000000 [14:05<00:00, 118216.28it/s]

leaderboard standings
------------------------------------
John Nelson           | 1428 | 0.980
Latonya Hong          | 1421 | 0.928
Robert Gonzalez       | 1418 | 0.956
Norma Poling          | 1417 | 0.971
Matthew Heuer         | 1415 | 0.917
Theresa Aitken        | 1411 | 0.952
Anthony Harbour       | 1407 | 0.911
Heather Poirrier      | 1406 | 0.972
Pasquale Brackett     | 1403 | 0.870
Joy Holstein          | 1401 | 0.935
Roy Pimentel          | 1397 | 0.995
William Henderson     | 1396 | 0.788
Martin Armstead       | 1394 | 0.965
Shannon Mcdonough     | 1391 | 0.995
Barbara Ellis         | 1391 | 0.917
Cara Driscoll         | 1389 | 0.919
Leon Stein            | 1389 | 0.956
Kellie Bucklin        | 1389 | 0.958
Jeffrey Brown         | 1388 | 0.884
Charla Rivera         | 1388 | 0.945
------------------------------------



In [424]:
p1 = player(2000, 0.999)
p2 = player(1900, 0.0)
draw_odds = (1 - abs(p1.skill_level - p2.skill_level)) * (p1.skill_level + p2.skill_level)
print("ratings: {}    -    {}".format(p1.rating, p2.rating))
print("skill:   {:.2f} - {:.2f} - {:.2f}".format(p1.skill_level, draw_odds, p2.skill_level))

for i in range(100):
    m = match(p1, p2, 32)
    result = m.simulate_match()
    if i % 10 == 0:
        print("---------------------")
        print("p1 {}  -  {} p2".format(result, 1-result))
        print("ratings: {} - {}".format(p1.rating, p2.rating))
        

ratings: 2000    -    1900
skill:   1.00 - 0.00 - 0.00
---------------------
p1 1  -  0 p2
ratings: 2012 - 1888
---------------------
p1 1  -  0 p2
ratings: 2090 - 1810
---------------------
p1 1  -  0 p2
ratings: 2134 - 1766
---------------------
p1 1  -  0 p2
ratings: 2164 - 1736
---------------------
p1 1  -  0 p2
ratings: 2185 - 1715
---------------------
p1 1  -  0 p2
ratings: 2205 - 1695
---------------------
p1 1  -  0 p2
ratings: 2219 - 1681
---------------------
p1 1  -  0 p2
ratings: 2229 - 1671
---------------------
p1 1  -  0 p2
ratings: 2239 - 1661
---------------------
p1 1  -  0 p2
ratings: 2249 - 1651
